In [1]:
from idealab_tools.kinematics.kinematics import Quaternion
from typing import Dict, Union
import mujoco
from gymnasium import spaces
import gymnasium as gym
import gymnasium
import numpy as np
import numpy
from gymnasium import utils
from gymnasium.spaces import Box
import math
import os
import os
import mujoco
import numpy
import mediapy as media
import matplotlib.pyplot as plt
import math
import yaml

In [2]:
width = 800
height = 600

In [3]:
with open('my_envs/biped.xml') as f:
    xml_template = f.read()

In [4]:
xml = xml_template.format(l_ankle=.03,k=.1,b=.01,ts=1e-3,width=800,height=600)

In [5]:
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model,width=width,height=height)
# renderer = mujoco.Renderer(model)


libdecor-gtk-WARNING: Failed to initialize GTK
Failed to load plugin 'libdecor-gtk.so': failed to init
No plugins found, falling back on no decorations


In [6]:
A = .25
d = 0 
ts = .25
f = .5
l1 = 0
l2 = .5

def mycontroller(model, data):
    t = data.time
    linear_act = t*d/ts
    if linear_act < d:
        linear_act = d
    pos1 = A*math.sin(2*math.pi*(f*t-l1))
    pos2 = A*math.sin(2*math.pi*(f*t-l2))
    data.ctrl = [pos1,pos2]
    return


# mujoco.set_mjcb_control(mycontroller)

In [7]:
mujoco.mj_resetData(model,data)
# data.qpos[0:2] = [-45*math.pi/180,-45*math.pi/180]
# data.qvel[0:2] = [-3,0]
frames=[]

duration = 5
framerate =30

q = []
w = []
t = []
xy = []

while data.time<duration:


    mujoco.mj_step(model,data)

    q.append(data.qpos.copy())
    w.append(data.qvel.copy())
    xy.append(data.xpos.copy())
    t.append(data.time)
    
    if len(frames)<data.time*framerate:
    
        # renderer.update_scene(data)
        renderer.update_scene(data,"world")
        pixels = renderer.render()
        frames.append(pixels)

mujoco.set_mjcb_control(None)
        
media.show_video(frames,fps = framerate,width=width,height=height)
# media.write_video('pendulum.mp4',frames,fps=framerate)

In [8]:
data.qpos

array([-1.64146693e-17,  2.85772734e-16,  1.04851401e-01,  1.00000000e+00,
        2.06891298e-17, -1.40187628e-16, -1.37490615e-18,  1.71760206e-17,
        9.43303875e-17,  4.53875623e-17, -1.87592674e-17,  5.58337033e-17,
        1.91909461e-16])

In [9]:
data.xpos

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.65184834e-17,  2.85710034e-16,  1.04851401e-01],
       [-1.08222735e-17,  2.00000000e-02,  8.48514015e-02],
       [-2.36804168e-19,  2.00000000e-02,  4.48514015e-02],
       [ 6.48148292e-18,  2.00000000e-02,  4.85140146e-03],
       [-1.09391811e-17, -2.00000000e-02,  8.48514015e-02],
       [ 1.07025913e-18, -2.00000000e-02,  4.48514015e-02],
       [ 1.08293988e-17, -2.00000000e-02,  4.85140146e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])